In [56]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [57]:
from pathlib import Path

root = "https://ai.stanford.edu/~amaas/data/sentiment/"
filename = "aclImdb_v1.tar.gz"
filepath = tf.keras.utils.get_file(filename, root + filename, extract=True, cache_dir="./imdb")
path = Path(filepath).with_name("aclImdb")
path

WindowsPath('/tmp/.keras/datasets/aclImdb')

In [58]:
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]

train_pos = review_paths(path / "train" / "pos")
train_neg = review_paths(path / "train" / "neg")
test_val_pos = review_paths(path / "test" / "pos")
test_val_neg = review_paths(path / "test" / "neg")

len(train_pos), len(train_neg), len(test_val_pos), len(test_val_neg)

(12500, 12500, 12500, 12500)

In [59]:
np.random.shuffle(test_val_pos)

test_pos = test_val_pos[:5000]
test_neg = test_val_neg[:5000]
valid_pos = test_val_pos[5000:]
valid_neg = test_val_neg[5000:]

In [66]:
def imdb_dataset(filepaths_positive, filepaths_negative):
    reviews = []
    labels = []
    for filepaths, label in ((filepaths_negative, 0), (filepaths_positive, 1)):
        for filepath in filepaths:
            with open(filepath, encoding="utf8") as review_file:
                reviews.append(review_file.read())
            labels.append(label)
    return tf.data.Dataset.from_tensor_slices((tf.constant(reviews), tf.constant(labels)))

In [67]:
batch_size = 32

train_set = imdb_dataset(train_pos, train_neg).shuffle(25000, seed=42)
train_set = train_set.batch(batch_size).prefetch(1)
valid_set = imdb_dataset(valid_pos, valid_neg).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).batch(batch_size).prefetch(1)